In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType

df= spark.read.table("workspace.bronze.erp_location")

**Triming**

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, F.trim(F.col(field.name)))


In [0]:
%sql
select * from workspace.bronze.erp_location

**CustomerID Correction**

In [0]:
df = df.withColumn("cid", F.regexp_replace(F.col("cid"), "-", ""))

**Normalisation**

In [0]:
df = df.withColumn(
    "cntry",
    F.when(F.col("cntry") == "DE", "Germany")
     .when(F.col("cntry").isin("US", "USA"), "United States")
     .when((F.col("cntry") == "") | F.col("cntry").isNull(), "n/a")
     .otherwise(F.col("cntry"))
)
     

**Rename Column**

In [0]:
RENAME_MAP = {
    "cid": "customer_number",
    "cntry": "country"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)

**Writing into silver table**

In [0]:
spark.sql("DROP TABLE IF EXISTS workspace.silver.erp_customer_location")
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.erp_customer_location")